 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="#Data-Preparation" data-toc-modified-id="Data-Preparation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Preparation</a></span></li><li><span><a href="#Match-ID" data-toc-modified-id="Match-ID-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Match ID</a></span></li><li><span><a href="#Parse-Email" data-toc-modified-id="Parse-Email-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Parse Email</a></span></li><li><span><a href="#Extract-words" data-toc-modified-id="Extract-words-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Extract words</a></span></li><li><span><a href="#Running-W2V" data-toc-modified-id="Running-W2V-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Running W2V</a></span></li></ul></div>

## Data Preparation

In [8]:
import pandas as pd

ImportError: No module named pandas

In [2]:
df = pd.read_csv("data/emails.csv")

In [3]:
df.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [4]:
print(df[1:2]['message'][1])

Message-ID: <15464986.1075855378456.JavaMail.evans@thyme>
Date: Fri, 4 May 2001 13:51:00 -0700 (PDT)
From: phillip.allen@enron.com
To: john.lavorato@enron.com
Subject: Re:
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: John J Lavorato <John J Lavorato/ENRON@enronXgate@ENRON>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst

Traveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepare a presentation.  I would suggest holding the business plan meetings here then take a trip without any formal business meetings.  I would even try and get some honest opinions on whether a trip is even desired or necessary.

As far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not.  Too often the

## Match ID

In [59]:
import csv, sys, os, re, glob

In [66]:
def list_files(dir):
    r = []
    for f in glob.iglob(dir + '/**/*.txt', recursive=True):
        usable_f = f.replace('\\','/')
        if re.match(r'[\w\d/.-]*3\.[0-9]*\.[A-Z0-9]*\.txt', usable_f):
            r.append(usable_f)
    return r

In [61]:
def is_empty_email(contents):
    empty_pattern = re.compile("X-ZLID:.*(\r|\n)*\*{11}[\r\n]EDRM Enron Email", re.MULTILINE)
    if empty_pattern.search(contents):
         return True
    return False

In [62]:
def match_trec(file):
#     to_pattern = r"To:.*<(.*)>"
    from_pattern = r"From:(.*)"
    contents_pattern = re.compile("X-ZLID:.*[\r\n]((?:.|\r|\n)*)\*{11}[\r\n]EDRM Enron Email", re.MULTILINE)
    from_name = file.split('/')[-3][14:-8]
    email_id = file.split('/')[-1][:-4]
    date_pattern = r"Date: (.*)"
    with open(file, 'r') as text:
        string = text.read()
#         if not is_empty_email(string):
        try:
#             to_string = re.search(to_pattern, string).group(1)
#                 from_string = re.search(from_pattern, string).group(1)
            contents = re.search(contents_pattern, string)
            if contents:
                contents = contents.group(1).replace('\n','').strip()
            else:
                contents = ''
#                 contents = re.search(contents_pattern, string).group(1).replace('\n','').strip()
            date_string = re.search(date_pattern, string).group(1).strip()
            return [from_name, date_string, contents, email_id]
        except:
            print('match_trec error:')
            print(file)
            print(string)
            print('-----------------')

In [67]:
working_path = '../edrmv2txt-v2/'

In [68]:
file_list = list_files(working_path)

KeyboardInterrupt: 

In [65]:
file_list[:10]

[]

In [110]:
from_names, dates, contents, email_ids = [], [], [], []
error_cnt = 0
no_info_cnt = 0
for f in file_list:
    trecid = f[:-4]
    try:
        info = match_trec(f)
        if info:
            from_names.append(info[0])
            dates.append(info[1])
            contents.append(info[2])
            email_ids.append(info[3])
        else:
            no_info_cnt += 1
    except UnicodeDecodeError:
        error_cnt += 1
print("files, errors, no info:",len(file_list),error_cnt, no_info_cnt)

files, errors, no info: 455449 331 0


In [112]:
contents = pd.DataFrame(
        {
            'id': email_ids,
            'sender': from_names,
            'dates': dates,
            'message': contents
        }
)

In [111]:
len(dates)

455118

In [113]:
contents

,dates,id,message,sender
0,"Thu, 10 May 2001 12:50:00 -0700 (PDT)",3.818885.JLRYP3ARLIIYUC4AX14ZJE3A1RAPULBFA,"Jeff,Jacques Craig will draw up a release. Wh...",allen-p
1,"Mon, 25 Sep 2000 07:01:00 -0700 (PDT)",3.818886.LDZQSGT5FQLDRLO1WBHN2MK2DFSDVTACA,---------------------- Forwarded by Phillip K ...,allen-p
2,"Wed, 9 May 2001 17:13:00 -0700 (PDT)",3.818896.IVOAU4ATG11DYDPDLW5BEB0LXJEL14TGB,The west desk would like 2 analysts.,allen-p
3,"Mon, 11 Sep 2000 09:19:00 -0700 (PDT)",3.818908.A0CV1HWH4CFTZMBCQWRRDZRJFIKDBFYJB,,allen-p
4,"Mon, 7 May 2001 16:23:00 -0700 (PDT)",3.818913.HRJLOFPA325QON43Q0PAAXW0NGI11IR2B,---------------------- Forwarded by Phillip K ...,allen-p
5,"Mon, 30 Apr 2001 13:37:00 -0700 (PDT)",3.818914.H0JO0RYAUKPP32YFJ0ICJQDTKFZ40POUA,---------------------- Forwarded by Phillip K ...,allen-p
6,"Tue, 14 Nov 2000 06:33:00 -0800 (PST)",3.818915.DVZB12GWTWPMSUO2NGZ31PLQBTMN2I3OB,"Ina, Where can we put Barry T.?Phillip--------...",allen-p
7,"Mon, 13 Nov 2000 05:25:00 -0800 (PST)",3.818917.LHBZH0YQ2IA3FFF1LTS23GR1VFGSQLDLB,"Faith, Regarding the 2001 plan, the members of...",allen-p
8,"Tue, 26 Sep 2000 09:27:00 -0700 (PDT)",3.818937.KUIKG5EH0XMXL4NLHHHTLNKELYHDH3FTB,,allen-p
9,"Tue, 19 Sep 2000 09:35:00 -0700 (PDT)",3.818952.KHTELI4FF4CEDZSFPHHGSCEBZ4E4O4SHB,---------------------- Forwarded by Phillip K ...,allen-p


In [114]:
contents.to_csv('parsed_id_message_date_sender.csv')

In [135]:
contents = pd.read_csv('parsed_id_message_date_sender.csv', encoding='latin1')

In [136]:
ids = pd.read_csv('data/qrels.txt', encoding='latin1', sep=r'\s', header=None, names=['id', 'stratum', 'label'])

/Accounts/nachtm/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [137]:
ids.head()

,id,stratum,label
0,401:3.10.DIE5L1APTGWLVTCIYE0KHRVQAV3CTGO5B,1000,-1
1,401:3.100.KK30VAAWQC5IX4YJBRVXMCMH3WL2Y5WAA,1000,-1
2,401:3.1000065.D2M301CFHC2HMASPQRHGTH3GT0JNYH3GA,1000,-1
3,401:3.1000065.D2M301CFHC2HMASPQRHGTH3GT0JNYH3GA.1,1000,-1
4,401:3.1000065.D2M301CFHC2HMASPQRHGTH3GT0JNYH3G...,1000,-1


In [138]:
contents.head()

,Unnamed: 0,dates,id,message,sender
0,0,"Thu, 10 May 2001 12:50:00 -0700 (PDT)",3.818885.JLRYP3ARLIIYUC4AX14ZJE3A1RAPULBFA,"Jeff,Jacques Craig will draw up a release. Wh...",allen-p
1,1,"Mon, 25 Sep 2000 07:01:00 -0700 (PDT)",3.818886.LDZQSGT5FQLDRLO1WBHN2MK2DFSDVTACA,---------------------- Forwarded by Phillip K ...,allen-p
2,2,"Wed, 9 May 2001 17:13:00 -0700 (PDT)",3.818896.IVOAU4ATG11DYDPDLW5BEB0LXJEL14TGB,The west desk would like 2 analysts.,allen-p
3,3,"Mon, 11 Sep 2000 09:19:00 -0700 (PDT)",3.818908.A0CV1HWH4CFTZMBCQWRRDZRJFIKDBFYJB,NaN,allen-p
4,4,"Mon, 7 May 2001 16:23:00 -0700 (PDT)",3.818913.HRJLOFPA325QON43Q0PAAXW0NGI11IR2B,---------------------- Forwarded by Phillip K ...,allen-p


In [140]:
def extract_senario(string):
    return string[:3]

In [141]:
def clean_id(string):
    return string[4:]

In [142]:
ids['scenario'] = ids.id.apply(extract_senario)

In [143]:
ids['id'] = ids.id.apply(clean_id)

In [144]:
'3.1000065.D2M301CFHC2HMASPQRHGTH3GT0JNYH3GA.6'.count('.')

3

In [145]:
test_pd = ids

In [146]:
test_pd = test_pd[test_pd['id'].map(lambda x: x.count('.')) == 2]

In [147]:
test_pd.shape

(1366347, 4)

In [148]:
ids = test_pd

In [149]:
to_join_left = ids[['id', 'label', 'scenario']]

In [150]:
to_join_left.head()

,id,label,scenario
0,3.10.DIE5L1APTGWLVTCIYE0KHRVQAV3CTGO5B,-1,401
1,3.100.KK30VAAWQC5IX4YJBRVXMCMH3WL2Y5WAA,-1,401
2,3.1000065.D2M301CFHC2HMASPQRHGTH3GT0JNYH3GA,-1,401
11,3.10001.KP0HSPM1ST2MRZ4RW04TK15BO4EN4UHGB,-1,401
12,3.1000114.BKEUF1CGPEZTNKAOAFIZ3YV0ZUJD3VZVB,-1,401


In [151]:
merged = pd.merge(to_join_left, contents)

In [165]:
merged.columns = ['ID','Label','Scenario','Garbage-ID','Date','Message-Contents','X-Origin']

In [167]:
del merged['Garbage-ID']

In [168]:
merged.head(100)

,ID,Label,Scenario,Date,Message-Contents,X-Origin
0,3.10.DIE5L1APTGWLVTCIYE0KHRVQAV3CTGO5B,-1,401,"Fri, 10 Nov 2000 07:00:00 -0800 (PST)",NaN,buy-r
1,3.10.DIE5L1APTGWLVTCIYE0KHRVQAV3CTGO5B,-1,402,"Fri, 10 Nov 2000 07:00:00 -0800 (PST)",NaN,buy-r
2,3.10.DIE5L1APTGWLVTCIYE0KHRVQAV3CTGO5B,-1,403,"Fri, 10 Nov 2000 07:00:00 -0800 (PST)",NaN,buy-r
3,3.100.KK30VAAWQC5IX4YJBRVXMCMH3WL2Y5WAA,-1,401,"Fri, 20 Oct 2000 01:00:00 -0700 (PDT)",NaN,buy-r
4,3.100.KK30VAAWQC5IX4YJBRVXMCMH3WL2Y5WAA,-1,402,"Fri, 20 Oct 2000 01:00:00 -0700 (PDT)",NaN,buy-r
5,3.100.KK30VAAWQC5IX4YJBRVXMCMH3WL2Y5WAA,-1,403,"Fri, 20 Oct 2000 01:00:00 -0700 (PDT)",NaN,buy-r
6,3.1000065.D2M301CFHC2HMASPQRHGTH3GT0JNYH3GA,-1,401,"Thu, 26 Oct 2000 09:43:00 -0700 (PDT)",Please see the attached articles:,kean-s_xml_
7,3.1000065.D2M301CFHC2HMASPQRHGTH3GT0JNYH3GA,-1,402,"Thu, 26 Oct 2000 09:43:00 -0700 (PDT)",Please see the attached articles:,kean-s_xml_
8,3.1000065.D2M301CFHC2HMASPQRHGTH3GT0JNYH3GA,0,403,"Thu, 26 Oct 2000 09:43:00 -0700 (PDT)",Please see the attached articles:,kean-s_xml_
9,3.10001.KP0HSPM1ST2MRZ4RW04TK15BO4EN4UHGB,-1,401,"Sun, 30 Nov 0002 00:00:00 -0800 (PST)",NaN,buy-r


In [153]:
ids.loc[ids['id'] == '3.818885.JLRYP3ARLIIYUC4AX14ZJE3A1RAPULBFA']

,id,stratum,label,scenario
595318,3.818885.JLRYP3ARLIIYUC4AX14ZJE3A1RAPULBFA,1000,-1,401
1280910,3.818885.JLRYP3ARLIIYUC4AX14ZJE3A1RAPULBFA,1000,-1,402
1966502,3.818885.JLRYP3ARLIIYUC4AX14ZJE3A1RAPULBFA,1000,-1,403


In [154]:
merged.loc[merged['id'] == '3.818885.JLRYP3ARLIIYUC4AX14ZJE3A1RAPULBFA']

,id,label,scenario,Unnamed: 0,dates,message,sender
1190649,3.818885.JLRYP3ARLIIYUC4AX14ZJE3A1RAPULBFA,-1,401,0,"Thu, 10 May 2001 12:50:00 -0700 (PDT)","Jeff,Jacques Craig will draw up a release. Wh...",allen-p
1190650,3.818885.JLRYP3ARLIIYUC4AX14ZJE3A1RAPULBFA,-1,402,0,"Thu, 10 May 2001 12:50:00 -0700 (PDT)","Jeff,Jacques Craig will draw up a release. Wh...",allen-p
1190651,3.818885.JLRYP3ARLIIYUC4AX14ZJE3A1RAPULBFA,-1,403,0,"Thu, 10 May 2001 12:50:00 -0700 (PDT)","Jeff,Jacques Craig will draw up a release. Wh...",allen-p


In [155]:
merged.to_csv('merged_id_label_senario_dates_message_sender.csv')

In [6]:
??pd.merge

In [143]:
gold_standard = merged.loc[merged['label'].isin([0,1])]
print(len(gold_standard))

9708


In [144]:
first = gold_standard.loc[gold_standard['scenario'] == '401']
second = gold_standard.loc[gold_standard['scenario'] == '402']
third = gold_standard.loc[gold_standard['scenario'] == '403']
print(len(first), len(second), len(third))

3244 3205 3259


In [ ]:
%cp merged_id_label_senario_dates_message_sender.csv data/

In [ ]:
mapping_dict = dict()

In [ ]:
meaning_ids = ids.loc[ids['label'].isin([0,1])]

In [ ]:
len(meaning_ids.loc[ids['id'] == 'apple']) is 0

In [158]:
merged.shape

(1365354, 7)

Kaggle Dataset

In [156]:
kaggle = pd.read_csv('data/emails.csv')

In [159]:
kaggle.head()

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [152]:
print(kaggle['message'][0])

Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>
Date: Mon, 14 May 2001 16:39:00 -0700 (PDT)
From: phillip.allen@enron.com
To: tim.belden@enron.com
Subject: 
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Tim Belden <Tim Belden/Enron@EnronXGate>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst

Here is our forecast

 


## Parse Email

In [45]:
def parse_metadata(data):
    data = data.split('\n')
    data = [d.split(': ') for d in data]
    valid_heads = ['Date', 'From', 'To', 'Subject', 'Mime-Version', 'Content-Type', 'Content-Transfer-Encoding', 'X-From', 'X-To', 'X-cc', 'X-bcc', 'X-Folder', 'X-Origin', 'X-FileName', 'Message-ID']
    parsed = ['' for _ in range (len(valid_heads))]
    for d in data:
        if len(d) > 1:
                if d[0] in valid_heads:
                    popped = d.pop(0)
                    parsed[valid_heads.index(popped)] = ": ".join(d)
    return parsed
            
        
    

In [50]:
def get_kaggle_metadata(message):
    message_pattern = r'([\w\W]*X-FileName:.*)([\w\W]*)'
    results = re.search(message_pattern, message)
    metadata = results.group(1).strip()
    contents = results.group(2).replace('\n','').strip()
    parsed_metadata = parse_metadata(metadata)
    parsed_metadata.append(contents)
    return parsed_metadata

In [51]:
metadata_and_message = kaggle.message.apply(get_kaggle_metadata).apply(pd.Series)

In [52]:
metadata_and_message

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,<18782981.1075855378110.JavaMail.evans@thyme>,Here is our forecast
1,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",phillip.allen@enron.com,john.lavorato@enron.com,Re:,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Allen-P,pallen (Non-Privileged).pst,<15464986.1075855378456.JavaMail.evans@thyme>,Traveling to have a business meeting takes the...
2,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Leah Van Arsdall,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,<24216240.1075855687451.JavaMail.evans@thyme>,test successful. way to go!!!
3,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",phillip.allen@enron.com,randall.gay@enron.com,,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Randall L Gay,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,<13505866.1075863688222.JavaMail.evans@thyme>,"Randy, Can you send me a schedule of the salar..."
4,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Greg Piper,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,<30922949.1075863688243.JavaMail.evans@thyme>,Let's shoot for Tuesday at 11:45.
5,"Thu, 31 Aug 2000 04:17:00 -0700 (PDT)",phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Greg Piper,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,<30965995.1075863688265.JavaMail.evans@thyme>,"Greg, How about either next Tuesday or Thursda..."
6,"Tue, 22 Aug 2000 07:44:00 -0700 (PDT)",phillip.allen@enron.com,"david.l.johnson@enron.com, john.shafer@enron.com",,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,"david.l.johnson@enron.com, John Shafer",,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,<16254169.1075863688286.JavaMail.evans@thyme>,Please cc the following distribution list with...
7,"Fri, 14 Jul 2000 06:59:00 -0700 (PDT)",phillip.allen@enron.com,joyce.teixeira@enron.com,Re: PRC review - phone calls,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Joyce Teixeira,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,<17189699.1075863688308.JavaMail.evans@thyme>,any morning between 10 and 11:30
8,"Tue, 17 Oct 2000 02:26:00 -0700 (PDT)",phillip.allen@enron.com,mark.scott@enron.com,Re: High Speed Internet Access,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Mark Scott,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,<20641191.1075855687472.JavaMail.evans@thyme>,1. login: pallen pw: ke9davis I don't think t...
9,"Mon, 16 Oct 2000 06:44:00 -0700 (PDT)",phillip.allen@enron.com,zimam@enron.com,FW: fixed forward or other Collar floor gas pr...,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,zimam@enron.com,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Allen-P,pallen.nsf,<30795301.1075855687494.JavaMail.evans@thyme>,---------------------- Forwarded by Phillip K ...


In [53]:
metadata_and_message.columns = ['Date', 'From', 'To', 'Subject', 'Mime-Version', 'Content-Type', 'Content-Transfer-Encoding', 'X-From', 'X-To', 'X-cc', 'X-bcc', 'X-Folder', 'X-Origin', 'X-FileName', 'Message-ID', 'Message-Contents']

In [173]:
metadata_and_message['X-Origin'] = metadata_and_message['X-Origin'].apply(str.lower)

In [174]:
metadata_and_message

,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,Message-ID,Message-Contents
0,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",allen-p,pallen (Non-Privileged).pst,<18782981.1075855378110.JavaMail.evans@thyme>,Here is our forecast
1,"Fri, 4 May 2001 13:51:00 -0700 (PDT)",phillip.allen@enron.com,john.lavorato@enron.com,Re:,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",allen-p,pallen (Non-Privileged).pst,<15464986.1075855378456.JavaMail.evans@thyme>,Traveling to have a business meeting takes the...
2,"Wed, 18 Oct 2000 03:00:00 -0700 (PDT)",phillip.allen@enron.com,leah.arsdall@enron.com,Re: test,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Leah Van Arsdall,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,allen-p,pallen.nsf,<24216240.1075855687451.JavaMail.evans@thyme>,test successful. way to go!!!
3,"Mon, 23 Oct 2000 06:13:00 -0700 (PDT)",phillip.allen@enron.com,randall.gay@enron.com,,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Randall L Gay,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,allen-p,pallen.nsf,<13505866.1075863688222.JavaMail.evans@thyme>,"Randy, Can you send me a schedule of the salar..."
4,"Thu, 31 Aug 2000 05:07:00 -0700 (PDT)",phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Greg Piper,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,allen-p,pallen.nsf,<30922949.1075863688243.JavaMail.evans@thyme>,Let's shoot for Tuesday at 11:45.
5,"Thu, 31 Aug 2000 04:17:00 -0700 (PDT)",phillip.allen@enron.com,greg.piper@enron.com,Re: Hello,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Greg Piper,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,allen-p,pallen.nsf,<30965995.1075863688265.JavaMail.evans@thyme>,"Greg, How about either next Tuesday or Thursda..."
6,"Tue, 22 Aug 2000 07:44:00 -0700 (PDT)",phillip.allen@enron.com,"david.l.johnson@enron.com, john.shafer@enron.com",,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,"david.l.johnson@enron.com, John Shafer",,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,allen-p,pallen.nsf,<16254169.1075863688286.JavaMail.evans@thyme>,Please cc the following distribution list with...
7,"Fri, 14 Jul 2000 06:59:00 -0700 (PDT)",phillip.allen@enron.com,joyce.teixeira@enron.com,Re: PRC review - phone calls,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Joyce Teixeira,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,allen-p,pallen.nsf,<17189699.1075863688308.JavaMail.evans@thyme>,any morning between 10 and 11:30
8,"Tue, 17 Oct 2000 02:26:00 -0700 (PDT)",phillip.allen@enron.com,mark.scott@enron.com,Re: High Speed Internet Access,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Mark Scott,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,allen-p,pallen.nsf,<20641191.1075855687472.JavaMail.evans@thyme>,1. login: pallen pw: ke9davis I don't think t...
9,"Mon, 16 Oct 2000 06:44:00 -0700 (PDT)",phillip.allen@enron.com,zimam@enron.com,FW: fixed forward or other Collar floor gas pr...,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,zimam@enron.com,,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,allen-p,pallen.nsf,<30795301.1075855687494.JavaMail.evans@thyme>,---------------------- Forwarded by Phillip K ...


In [175]:
metadata_and_message.shape

(517401, 16)

In [319]:
metadata_and_message.loc[metadata_and_message['message'] == 'Here is our forecast'].loc[277]['metadata']

"Message-ID: <4070589.1075855723617.JavaMail.evans@thyme>\nDate: Mon, 14 May 2001 06:39:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: tim.belden@enron.com\nSubject: \nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Phillip K Allen\nX-To: Tim Belden\nX-cc: \nX-bcc: \nX-Folder: \\Phillip_Allen_June2001\\Notes Folders\\'sent mail\nX-Origin: Allen-P\nX-FileName: pallen.nsf"

In [179]:
merged_with_kaggle = pd.merge(metadata_and_message, merged, how='inner', on=['Message-Contents','X-Origin','Date'], sort=False)

In [201]:
deduped_full_dataset = merged_with_kaggle.drop_duplicates(subset=['ID', 'Scenario', 'Message-Contents'], keep = 'first')

In [203]:
deduped_full_dataset.shape

(570234, 19)

In [206]:
deduped_full_dataset.head()

,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,Message-ID,Message-Contents,ID,Label,Scenario
0,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",allen-p,pallen (Non-Privileged).pst,<18782981.1075855378110.JavaMail.evans@thyme>,Here is our forecast,3.819795.JW4ITC4HSNF5VL3Q14VSPS2RHJ2AWAEMB,-1,401
1,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",allen-p,pallen (Non-Privileged).pst,<18782981.1075855378110.JavaMail.evans@thyme>,Here is our forecast,3.819795.JW4ITC4HSNF5VL3Q14VSPS2RHJ2AWAEMB,-1,402
2,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",allen-p,pallen (Non-Privileged).pst,<18782981.1075855378110.JavaMail.evans@thyme>,Here is our forecast,3.819795.JW4ITC4HSNF5VL3Q14VSPS2RHJ2AWAEMB,-1,403
3,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",allen-p,pallen (Non-Privileged).pst,<18782981.1075855378110.JavaMail.evans@thyme>,Here is our forecast,3.824632.IK2LQDKGTGEH1KM5OAHT3QA5G5BVZ5F1B,0,401
4,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",allen-p,pallen (Non-Privileged).pst,<18782981.1075855378110.JavaMail.evans@thyme>,Here is our forecast,3.824632.IK2LQDKGTGEH1KM5OAHT3QA5G5BVZ5F1B,0,402


In [197]:
merged_with_kaggle.to_csv('full_dataset.csv')

In [184]:
merged_with_kaggle_gs = merged_with_kaggle.loc[merged_with_kaggle['Label'].isin([0,1])]

In [208]:
deduped_full_dataset_gs = deduped_full_dataset.loc[merged_with_kaggle['Label'].isin([0,1])]

In [198]:
merged_with_kaggle_gs.to_csv('full_gold_standard.csv')

In [187]:
one = merged_with_kaggle_gs.loc[merged_with_kaggle_gs['Scenario'] == '401']
two =  merged_with_kaggle_gs.loc[merged_with_kaggle_gs['Scenario'] == '402']
three =  merged_with_kaggle_gs.loc[merged_with_kaggle_gs['Scenario'] == '403']
print(one.shape)
print(two.shape)
print(three.shape)

(3401, 19)
(3331, 19)
(3342, 19)


In [209]:
one = deduped_full_dataset_gs.loc[deduped_full_dataset_gs['Scenario'] == '401']
two =  deduped_full_dataset_gs.loc[deduped_full_dataset_gs['Scenario'] == '402']
three =  deduped_full_dataset_gs.loc[deduped_full_dataset_gs['Scenario'] == '403']
print(one.shape)
print(two.shape)
print(three.shape)

(1800, 19)
(1697, 19)
(1615, 19)


In [212]:
print(one.loc[one['Label'] == 1].shape)
print(two.loc[two['Label'] == 1].shape)
print(three.loc[three['Label'] == 1].shape)

(826, 19)
(320, 19)
(166, 19)


,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,Message-ID,Message-Contents,ID,Label,Scenario
9,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\PALLEN (Non-Privileged)\Allen, Phillip K.\Sen...",allen-p,PALLEN (Non-Privileged).pst,<12792381.1075858637829.JavaMail.evans@thyme>,Here is our forecast,3.824632.IK2LQDKGTGEH1KM5OAHT3QA5G5BVZ5F1B,0,401
10,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\PALLEN (Non-Privileged)\Allen, Phillip K.\Sen...",allen-p,PALLEN (Non-Privileged).pst,<12792381.1075858637829.JavaMail.evans@thyme>,Here is our forecast,3.824632.IK2LQDKGTGEH1KM5OAHT3QA5G5BVZ5F1B,0,402
11,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,,,"\PALLEN (Non-Privileged)\Allen, Phillip K.\Sen...",allen-p,PALLEN (Non-Privileged).pst,<12792381.1075858637829.JavaMail.evans@thyme>,Here is our forecast,3.824632.IK2LQDKGTGEH1KM5OAHT3QA5G5BVZ5F1B,0,403
87,"Tue, 17 Oct 2000 02:26:00 -0700 (PDT)",phillip.allen@enron.com,mark.scott@enron.com,Re: High Speed Internet Access,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Mark Scott,,,\Phillip_Allen_Dec2000\Notes Folders\All docum...,allen-p,pallen.nsf,<13486797.1075855667594.JavaMail.evans@thyme>,1. login: pallen pw: ke9davis I don't think t...,3.819904.FD4XODPI03CAHPT0VK55U3DHJ0DE2RO3A,0,401
88,"Tue, 17 Oct 2000 02:26:00 -0700 (PDT)",phillip.allen@enron.com,mark.scott@enron.com,Re: High Speed Internet Access,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Mark Scott,,,\Phillip_Allen_Dec2000\Notes Folders\All docum...,allen-p,pallen.nsf,<13486797.1075855667594.JavaMail.evans@thyme>,1. login: pallen pw: ke9davis I don't think t...,3.819904.FD4XODPI03CAHPT0VK55U3DHJ0DE2RO3A,0,402
89,"Tue, 17 Oct 2000 02:26:00 -0700 (PDT)",phillip.allen@enron.com,mark.scott@enron.com,Re: High Speed Internet Access,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Mark Scott,,,\Phillip_Allen_Dec2000\Notes Folders\All docum...,allen-p,pallen.nsf,<13486797.1075855667594.JavaMail.evans@thyme>,1. login: pallen pw: ke9davis I don't think t...,3.819904.FD4XODPI03CAHPT0VK55U3DHJ0DE2RO3A,0,403
90,"Tue, 17 Oct 2000 02:26:00 -0700 (PDT)",phillip.allen@enron.com,mark.scott@enron.com,Re: High Speed Internet Access,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Mark Scott,,,\Phillip_Allen_Dec2000\Notes Folders\Sent,allen-p,pallen.nsf,<6212311.1075855680932.JavaMail.evans@thyme>,1. login: pallen pw: ke9davis I don't think t...,3.819904.FD4XODPI03CAHPT0VK55U3DHJ0DE2RO3A,0,401
91,"Tue, 17 Oct 2000 02:26:00 -0700 (PDT)",phillip.allen@enron.com,mark.scott@enron.com,Re: High Speed Internet Access,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Mark Scott,,,\Phillip_Allen_Dec2000\Notes Folders\Sent,allen-p,pallen.nsf,<6212311.1075855680932.JavaMail.evans@thyme>,1. login: pallen pw: ke9davis I don't think t...,3.819904.FD4XODPI03CAHPT0VK55U3DHJ0DE2RO3A,0,402
92,"Tue, 17 Oct 2000 02:26:00 -0700 (PDT)",phillip.allen@enron.com,mark.scott@enron.com,Re: High Speed Internet Access,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Mark Scott,,,\Phillip_Allen_Dec2000\Notes Folders\Sent,allen-p,pallen.nsf,<6212311.1075855680932.JavaMail.evans@thyme>,1. login: pallen pw: ke9davis I don't think t...,3.819904.FD4XODPI03CAHPT0VK55U3DHJ0DE2RO3A,0,403
129,"Mon, 9 Oct 2000 07:16:00 -0700 (PDT)",phillip.allen@enron.com,keith.holst@enron.com,Consolidated positions: Issues & To Do list,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Keith Holst,,,\Phillip_Allen_Dec2000\Notes Folders\All docum...,allen-p,pallen.nsf,<7452188.1075855667684

In [195]:
merged_with_kaggle_gs.loc[merged_with_kaggle_gs['ID']=='3.824632.IK2LQDKGTGEH1KM5OAHT3QA5G5BVZ5F1B']['X-Folder'][9]

'\\PALLEN (Non-Privileged)\\Allen, Phillip K.\\Sent Items'

In [196]:
merged_with_kaggle_gs.loc[merged_with_kaggle_gs['ID']=='3.824632.IK2LQDKGTGEH1KM5OAHT3QA5G5BVZ5F1B']['X-Folder'][3]

"\\Phillip_Allen_Jan2002_1\\Allen, Phillip K.\\'Sent Mail"

In [1]:
from datetime import datetime
import pandas as pd

In [2]:
data = pd.read_csv('./full_dataset.csv')
data.head()

/Accounts/nachtm/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,Message-ID,Message-Contents,ID,Label,Scenario
0,0,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,NaN,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,NaN,NaN,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",allen-p,pallen (Non-Privileged).pst,<18782981.1075855378110.JavaMail.evans@thyme>,Here is our forecast,3.819795.JW4ITC4HSNF5VL3Q14VSPS2RHJ2AWAEMB,-1,401
1,1,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,NaN,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,NaN,NaN,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",allen-p,pallen (Non-Privileged).pst,<18782981.1075855378110.JavaMail.evans@thyme>,Here is our forecast,3.819795.JW4ITC4HSNF5VL3Q14VSPS2RHJ2AWAEMB,-1,402
2,2,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,NaN,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,NaN,NaN,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",allen-p,pallen (Non-Privileged).pst,<18782981.1075855378110.JavaMail.evans@thyme>,Here is our forecast,3.819795.JW4ITC4HSNF5VL3Q14VSPS2RHJ2AWAEMB,-1,403
3,3,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,NaN,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,NaN,NaN,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",allen-p,pallen (Non-Privileged).pst,<18782981.1075855378110.JavaMail.evans@thyme>,Here is our forecast,3.824632.IK2LQDKGTGEH1KM5OAHT3QA5G5BVZ5F1B,0,401
4,4,"Mon, 14 May 2001 16:39:00 -0700 (PDT)",phillip.allen@enron.com,tim.belden@enron.com,NaN,1.0,text/plain; charset=us-ascii,7bit,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,NaN,NaN,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",allen-p,pallen (Non-Privileged).pst,<18782981.1075855378110.JavaMail.evans@thyme>,Here is our forecast,3.824632.IK2LQDKGTGEH1KM5OAHT3QA5G5BVZ5F1B,0,402


In [3]:
deduped_full_dataset = data.drop_duplicates(subset=['ID', 'Scenario', 'Message-Contents'], keep = 'first')

In [4]:
deduped_full_dataset.to_csv('./data/full_dataset.csv')

## Create test and training sets

In [3]:
import pandas as pd

In [4]:
full_data = pd.read_csv('./data/parsed/full_dataset.csv')
gold_standard = pd.read_csv('./data/parsed/full_gold_standard.csv')

/home/dan/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
570234 * .7

399163.8

In [5]:
full_seventy = full_data[:399165]
full_thirty = full_data[399165:]

In [11]:
full_thirty_gs = full_thirty.loc[full_thirty['Label'].isin([0,1])]
full_thirty_gs.shape

(1445, 21)

In [12]:
full_seventy_gs = full_seventy.loc[full_seventy['Label'].isin([0,1])]
full_seventy_gs.shape

(3667, 21)

In [16]:
full_seventy.to_csv('./data/parsed/training.csv')
full_thirty.to_csv('./data/parsed/test.csv')

## Parse metadata

In [ ]:
#CHANGE TO FUNCTION AND USE APPLYMAP??
for i, d in enumerate(data['Date']):
    d = d.replace(',', '')
    redundancy_filter = d.split(' (')
    string = redundancy_filter[0]
    datetime_object = datetime.strptime(string, '%a %d %b %Y %X %z')
    data.set_value(i, 'Data', datetime_object) 
    data.head()

## Extract words

In [213]:
def extract_message(string):
    start = string.find('FileName:')
    sub = string[start: ].find('\n')
    return string[start: ][sub:]

In [214]:
df['content'] = df.message.apply(extract_message)

The history saving thread hit an unexpected error (OperationalError('disk I/O error',)).History will not be written to the database.


KeyboardInterrupt: 

In [ ]:
df.content.head(5)

In [ ]:
import re
def getWords(text):
    return " ".join(re.compile('\w*[A-Za-z]\w*').findall(text))

In [ ]:
from nltk.tokenize import TreebankWordTokenizer
def token(text):
    tokenizer = TreebankWordTokenizer()
    return tokenizer.tokenize(text)

In [ ]:
df_test = df[1:50]

In [ ]:
df_test['words'] = df_test.content.apply(getWords)

In [ ]:
df_test['words'] = df_test.content.apply(token)

In [ ]:
df_test.words.head()

In [ ]:
df['words'] = df.content.apply(getWords)

In [ ]:
df.words.head()

In [ ]:
df_shuffled = df.sample(frac=1).reset_index(drop=True)

In [ ]:
test = df_shuffled[0:1000]

## Running W2V

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [ ]:
n_features = 1000

In [ ]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(test.words)

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
n_topics = 10
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=50,
                                learning_method='batch',
                                learning_offset=50.,
                                random_state=42)

In [ ]:
lda.fit(tf)

In [ ]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [ ]:
n_top_words = 20

In [ ]:
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

In [ ]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)